In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Advanced").getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/25 17:48:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/25 17:48:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/03/25 17:48:31 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


exploratory data analysis

In [4]:
df = spark.read.json('/Users/rob_the_programmer/Documents/programming/2022/hands_on_data_science/spark-sql-dataframes-workspace/files/utlization.json')

In [6]:
df.show(5)

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|
+---------------+-------------------+-----------+---------+-------------+
only showing top 5 rows



In [17]:
df.createOrReplaceTempView('utilization')

In [18]:
df.count()

500000

In [19]:
df.describe().show()

+-------+-------------------+-------------------+-------------------+------------------+------------------+
|summary|    cpu_utilization|     event_datetime|        free_memory|         server_id|     session_count|
+-------+-------------------+-------------------+-------------------+------------------+------------------+
|  count|             500000|             500000|             500000|            500000|            500000|
|   mean| 0.6205177400000106|               null| 0.3791280999999971|             124.5|          69.59616|
| stddev|0.15875173872912937|               null|0.15830931278376237|14.430884120552962|14.850676696352718|
|    min|               0.22|03/05/2019 08:06:14|                0.0|               100|                32|
|    max|                1.0|04/09/2019 01:22:46|               0.78|               149|               105|
+-------+-------------------+-------------------+-------------------+------------------+------------------+



In [20]:
df.describe('cpu_utilization', 'free_memory').show()

+-------+-------------------+-------------------+
|summary|    cpu_utilization|        free_memory|
+-------+-------------------+-------------------+
|  count|             500000|             500000|
|   mean| 0.6205177400000106| 0.3791280999999971|
| stddev|0.15875173872912937|0.15830931278376237|
|    min|               0.22|                0.0|
|    max|                1.0|               0.78|
+-------+-------------------+-------------------+



In [21]:
df.stat.corr('cpu_utilization', 'free_memory')

-0.47047715730807543

In [22]:
df.stat.freqItems(['server_id', 'session_count']).show()

+--------------------+-----------------------+
| server_id_freqItems|session_count_freqItems|
+--------------------+-----------------------+
|[146, 137, 101, 1...|   [92, 101, 83, 104...|
+--------------------+-----------------------+



In [23]:
dfsample = df.sample(fraction=0.05, withReplacement=False)
dfsample.count()

24764

In [24]:
spark.sql('''
    select 
        min(cpu_utilization) as min_cpu_utilization,
        max(cpu_utilization) as max_cpu_utilization,
        stddev(cpu_utilization) as stddev_cpu_utilization
    from utilization
''').show()

+-------------------+-------------------+----------------------+
|min_cpu_utilization|max_cpu_utilization|stddev_cpu_utilization|
+-------------------+-------------------+----------------------+
|               0.22|                1.0|   0.15875173872912937|
+-------------------+-------------------+----------------------+



In [25]:
spark.sql('''
    select 
        server_id,
        min(cpu_utilization) as min_cpu_utilization,
        max(cpu_utilization) as max_cpu_utilization,
        stddev(cpu_utilization) as stddev_cpu_utilization
    from utilization
    group by 1
''').show()

+---------+-------------------+-------------------+----------------------+
|server_id|min_cpu_utilization|max_cpu_utilization|stddev_cpu_utilization|
+---------+-------------------+-------------------+----------------------+
|      112|               0.52|               0.92|   0.11528867845082576|
|      113|               0.58|               0.98|   0.11544345150353687|
|      110|               0.35|               0.75|   0.11533251724450215|
|      107|               0.45|               0.85|   0.11597417369783877|
|      103|               0.56|               0.96|   0.11617507884178278|
|      104|               0.51|               0.91|   0.11521679513850511|
|      106|               0.22|               0.62|   0.11531539914568233|
|      111|               0.36|               0.76|   0.11530221569464506|
|      100|               0.27|               0.67|    0.1152264191787964|
|      105|               0.29|               0.69|   0.11510721467869486|
|      108|              

In [26]:
spark.sql('''
    select
        server_id,
        floor(cpu_utilization * 100/10) bucket 
    from utilization''').show()

+---------+------+
|server_id|bucket|
+---------+------+
|      100|     5|
|      100|     4|
|      100|     5|
|      100|     5|
|      100|     3|
|      100|     4|
|      100|     5|
|      100|     4|
|      100|     5|
|      100|     5|
|      100|     3|
|      100|     6|
|      100|     6|
|      100|     5|
|      100|     2|
|      100|     4|
|      100|     4|
|      100|     6|
|      100|     4|
|      100|     5|
+---------+------+
only showing top 20 rows



In [29]:
spark.sql('''
    select
        count(*),
        floor(cpu_utilization * 100/10) as bucket 
    from utilization
    group by 2
    order by 2''').show()

+--------+------+
|count(1)|bucket|
+--------+------+
|    8186|     2|
|   37029|     3|
|   68046|     4|
|  104910|     5|
|  116725|     6|
|   88242|     7|
|   56598|     8|
|   20207|     9|
|      57|    10|
+--------+------+



In [32]:
spark.sql('''
    select
        server_id,
        min(cpu_utilization) as min_cpu_utilization,
        max(cpu_utilization) as max_cpu_utilization,
        stddev(cpu_utilization) as stddev_cpu_utilization
    from utilization
    group by 1
    ''').show()

+---------+-------------------+-------------------+----------------------+
|server_id|min_cpu_utilization|max_cpu_utilization|stddev_cpu_utilization|
+---------+-------------------+-------------------+----------------------+
|      112|               0.52|               0.92|   0.11528867845082576|
|      113|               0.58|               0.98|   0.11544345150353687|
|      110|               0.35|               0.75|   0.11533251724450215|
|      107|               0.45|               0.85|   0.11597417369783877|
|      103|               0.56|               0.96|   0.11617507884178278|
|      104|               0.51|               0.91|   0.11521679513850511|
|      106|               0.22|               0.62|   0.11531539914568233|
|      111|               0.36|               0.76|   0.11530221569464506|
|      100|               0.27|               0.67|    0.1152264191787964|
|      105|               0.29|               0.69|   0.11510721467869486|
|      108|              

In [34]:
sqlwindow = spark.sql('''
    select
        event_datetime,
        server_id,
        cpu_utilization,
        avg(cpu_utilization)
        over (
            partition by server_id
        ) as avg_server_util
    from utilization
    ''')
    
sqlwindow.show()

+-------------------+---------+---------------+------------------+
|     event_datetime|server_id|cpu_utilization|   avg_server_util|
+-------------------+---------+---------------+------------------+
|03/05/2019 08:06:31|      110|           0.68|0.5537749999999892|
|03/05/2019 08:11:31|      110|           0.58|0.5537749999999892|
|03/05/2019 08:16:31|      110|           0.55|0.5537749999999892|
|03/05/2019 08:21:31|      110|           0.63|0.5537749999999892|
|03/05/2019 08:26:31|      110|           0.63|0.5537749999999892|
|03/05/2019 08:31:31|      110|           0.71|0.5537749999999892|
|03/05/2019 08:36:31|      110|           0.67|0.5537749999999892|
|03/05/2019 08:41:31|      110|           0.55|0.5537749999999892|
|03/05/2019 08:46:31|      110|           0.37|0.5537749999999892|
|03/05/2019 08:51:31|      110|            0.7|0.5537749999999892|
|03/05/2019 08:56:31|      110|           0.67|0.5537749999999892|
|03/05/2019 09:01:31|      110|           0.56|0.5537749999999

In [35]:
sqlwindow2 = spark.sql('''
    select
        event_datetime,
        server_id,
        cpu_utilization,
        avg(cpu_utilization)
        over (
            partition by server_id
        ) as avg_server_util,
        cpu_utilization - avg(cpu_utilization) 
        over (
            partition by server_id
        ) as delta_server_util
    from utilization
    ''')
    
sqlwindow2.show()

+-------------------+---------+---------------+------------------+--------------------+
|     event_datetime|server_id|cpu_utilization|   avg_server_util|   delta_server_util|
+-------------------+---------+---------------+------------------+--------------------+
|03/05/2019 08:06:31|      110|           0.68|0.5537749999999892|  0.1262250000000108|
|03/05/2019 08:11:31|      110|           0.58|0.5537749999999892|0.026225000000010712|
|03/05/2019 08:16:31|      110|           0.55|0.5537749999999892|-0.00377499999998...|
|03/05/2019 08:21:31|      110|           0.63|0.5537749999999892| 0.07622500000001076|
|03/05/2019 08:26:31|      110|           0.63|0.5537749999999892| 0.07622500000001076|
|03/05/2019 08:31:31|      110|           0.71|0.5537749999999892| 0.15622500000001072|
|03/05/2019 08:36:31|      110|           0.67|0.5537749999999892| 0.11622500000001079|
|03/05/2019 08:41:31|      110|           0.55|0.5537749999999892|-0.00377499999998...|
|03/05/2019 08:46:31|      110| 

In [36]:
sqlwindow3 = spark.sql('''
    select
        event_datetime,
        server_id,
        cpu_utilization,
        avg(cpu_utilization)
        over (
            partition by server_id
            order by 1
            rows between 1 preceding and 1 following
        ) as avg_server_util
    from utilization
    ''')
    
sqlwindow3.show()

+-------------------+---------+---------------+-------------------+
|     event_datetime|server_id|cpu_utilization|    avg_server_util|
+-------------------+---------+---------------+-------------------+
|03/05/2019 08:06:31|      110|           0.68|               0.63|
|03/05/2019 08:11:31|      110|           0.58| 0.6033333333333334|
|03/05/2019 08:16:31|      110|           0.55| 0.5866666666666666|
|03/05/2019 08:21:31|      110|           0.63| 0.6033333333333334|
|03/05/2019 08:26:31|      110|           0.63| 0.6566666666666666|
|03/05/2019 08:31:31|      110|           0.71| 0.6699999999999999|
|03/05/2019 08:36:31|      110|           0.67| 0.6433333333333333|
|03/05/2019 08:41:31|      110|           0.55| 0.5300000000000001|
|03/05/2019 08:46:31|      110|           0.37|               0.54|
|03/05/2019 08:51:31|      110|            0.7|               0.58|
|03/05/2019 08:56:31|      110|           0.67| 0.6433333333333334|
|03/05/2019 09:01:31|      110|           0.56| 